In [2]:
import os
import cv2
import argparse
import json
import numpy as np
import pandas as pd
import shutil

from tqdm import tqdm
from os import path, walk, makedirs
from skimage.io import imread
from shapely import wkt
from shapely.geometry import mapping, MultiPolygon, Polygon
from cv2 import fillPoly, imwrite
from PIL import Image
from shapely.geometry import MultiPolygon, Polygon
import shapely.wkt
import matplotlib.pyplot as plt

In [3]:
print("Current Dir",os.getcwd())
os.chdir('../..')
print("Current Dir",os.getcwd())

Current Dir /Users/yaminigotimukul/DataScience/Berekley/Semesters/Spring_2024/repo/alivio/src/03_data_engineering
Current Dir /Users/yaminigotimukul/DataScience/Berekley/Semesters/Spring_2024/repo/alivio


In [11]:
ROOT_DIR=os.getcwd()
BASE_DATA_DIR = os.path.join(ROOT_DIR, 'data')
print("BASE_DIR : ", BASE_DATA_DIR)
DATA_DIR = os.path.join(BASE_DATA_DIR, 'xview_building_damage/train/sample')
print("DATA_DIR : ", DATA_DIR)
metadata_csv_path=os.path.join(BASE_DATA_DIR, 'xview_building_damage/train/xview2_processed.csv')
xbd_train = os.path.join(BASE_DATA_DIR, 'xview_building_damage', 'train', 'clean_meta_csv')

ROOT_DATA_DIR = os.path.join(BASE_DATA_DIR, 'xview_building_damage')
print("Root Data Path : ", ROOT_DATA_DIR)
CHALLENGE_DIR = os.path.join(ROOT_DATA_DIR, 'challenge')
TRAIN_DIR=os.path.join(CHALLENGE_DIR, 'train')
HOLD_DIR=os.path.join(CHALLENGE_DIR, 'hold')
TEST_DIR=os.path.join(CHALLENGE_DIR, 'test')

train_out_dir=os.path.join(TRAIN_DIR, 'disaster')
test_out_dir=os.path.join(TEST_DIR, 'disaster')
hold_out_dir=os.path.join(HOLD_DIR, 'disaster')

BASE_DIR :  /Users/yaminigotimukul/DataScience/Berekley/Semesters/Spring_2024/repo/alivio/data
DATA_DIR :  /Users/yaminigotimukul/DataScience/Berekley/Semesters/Spring_2024/repo/alivio/data/xview_building_damage/train/sample
Root Data Path :  /Users/yaminigotimukul/DataScience/Berekley/Semesters/Spring_2024/repo/alivio/data/xview_building_damage


### Metafiles Cleanup Code

In [7]:
def get_df_with_class_numeric_labels(df_name):
    df_name['damage'].fillna('pre', inplace=True)
    df_name['damage_class']=df_name['damage']
    keys=list(df_name['damage_class'].value_counts().keys())
    df_name['damage_class']=df_name['damage_class'].apply(keys.index)
    df_name['damage_class'].value_counts()
    return df_name

In [8]:
def pre_process_meta_data(meta_df, out_dir, disaster_name ='hurricane-florence', dataset='train'):
    hc_disaster = meta_df[meta_df['disaster']==disaster_name]
    hc_train = hc_disaster[hc_disaster['dataset']== dataset]
    df_disaster = hc_train[hc_train['image_polygon'].notna()]
    df_disaster['mask_file_names'] = df_disaster['img_name'].str.replace('.png', '_')+df_disaster['building_id']+'.png'
    df_disaster_class_labels = get_df_with_class_numeric_labels(df_disaster)
    df_disaster_class_labels.to_csv(out_dir+'/hc_'+dataset+"_"+disaster_name+".csv")

In [9]:
df_all = pd.read_csv(metadata_csv_path)

In [ ]:
### Michael
pre_process_meta_data(df_all, train_out_dir, 'hurricane-michael', 'train')
pre_process_meta_data(df_all, test_out_dir, 'hurricane-michael', 'test')
pre_process_meta_data(df_all, hold_out_dir, 'hurricane-michael', 'hold')

In [ ]:
### Florence
pre_process_meta_data(df_all, train_out_dir, 'hurricane-florence', 'train')
pre_process_meta_data(df_all, test_out_dir, 'hurricane-florence', 'test')
pre_process_meta_data(df_all, hold_out_dir, 'hurricane-florence', 'hold')

In [ ]:
### Harvey
pre_process_meta_data(df_all, train_out_dir, 'hurricane-harvey', 'train')
pre_process_meta_data(df_all, test_out_dir, 'hurricane-harvey', 'test')
pre_process_meta_data(df_all, hold_out_dir, 'hurricane-harvey', 'hold')

In [ ]:
### Matthew
pre_process_meta_data(df_all, train_out_dir, 'hurricane-matthew', 'train')
pre_process_meta_data(df_all, test_out_dir, 'hurricane-matthew', 'test')
pre_process_meta_data(df_all, hold_out_dir, 'hurricane-matthew', 'hold')

In [29]:
#!python src/utils/mask_generators/mask_polygons.py --input data/xview_building_damage/challenge/output --single-file --border 2
### Adapting the contents of the mask_polygons.py below
def get_dimensions(file_path):
    """
    :param file_path: The path of the file 
    :return: returns (width,height,channels)
    """
    # Open the image we are going to mask
    pil_img = imread(file_path)
    img = np.array(pil_img)
    w, h, c = img.shape
    return (w, h, c)

def mask_polygons_separately(size, shapes):
    """
    :param size: A tuple of the (width,height,channels)
    :param shapes: A list of points in the polygon from get_feature_info
    :returns: a dict of masked polygons with the shapes filled in from cv2.fillPoly
    """
    # For each WKT polygon, read the WKT format and fill the polygon as an image
    masked_polys = {}

    for u in shapes:
        sh = shapes[u]
        mask_img = np.zeros(size, np.uint8)
        i = fillPoly(mask_img, [sh], (255, 255, 255))
        masked_polys[u] = i

    return masked_polys

def mask_polygons_together(size, shapes):
    """
    :param size: A tuple of the (width,height,channels)
    :param shapes: A list of points in the polygon from get_feature_info
    :returns: A numpy array with the polygons filled 255s where there's a building and 0 where not 
    """
    # For each WKT polygon, read the WKT format and fill the polygon as an image
    mask_img = np.zeros(size, np.uint8)

    for u in shapes:
        blank =  np.zeros(size, np.uint8)
        poly = shapes[u] 
        fillPoly(blank, [poly], (1, 1, 1))
        mask_img += blank
    
    # Here we are taking the overlap (+=) and squashing it back to 0
    mask_img[mask_img > 1] = 0

    # Finally we are taking all 1s and making it pure white (255)
    mask_img[mask_img == 1] = 255

    return mask_img

def mask_polygons_together_with_border(size, shapes, border):
    """
    :param size: A tuple of the (width,height,channels)
    :param shapes: A list of points in the polygon from get_feature_info
    :returns: a dict of masked polygons with the shapes filled in from cv2.fillPoly
    """

    # For each WKT polygon, read the WKT format and fill the polygon as an image
    mask_img = np.zeros(size, np.uint8)

    for u in shapes:
        blank =  np.zeros(size, np.uint8)
        # Each polygon stored in shapes is a np.ndarray
        poly = shapes[u]
        
        # Creating a shapely polygon object out of the numpy array 
        polygon = Polygon(poly)

        # Getting the center points from the polygon and the polygon points
        (poly_center_x, poly_center_y) = polygon.centroid.coords[0]
        polygon_points = polygon.exterior.coords

        # Setting a new polygon with each X,Y manipulated based off the center point
        shrunk_polygon = []
        for (x,y) in polygon_points:
            if x < poly_center_x:
                x += border
            elif x > poly_center_x:
                x -= border

            if y < poly_center_y:
                y += border
            elif y > poly_center_y:
                y -= border

            shrunk_polygon.append([x,y])
        
        # Transforming the polygon back to a np.ndarray
        ns_poly = np.array(shrunk_polygon, np.int32)
  
        # Filling the shrunken polygon to add a border between close polygons
        fillPoly(blank, [ns_poly], (1, 1, 1))
        mask_img += blank
    
    mask_img[mask_img > 1] = 0
    mask_img[mask_img == 1] = 255
    return mask_img

def save_masks(masks, output_path, mask_file_name):
    """
    :param masks: dictionary of UID:masked polygons from mask_polygons_separately()
    :param output_path: path to save the masks
    :param mask_file_name: the file name the masks should have 
    """
    # For each filled polygon, write out a separate file, increasing the name
    for m in masks:
        final_out = path.join(output_path,
                              mask_file_name + '_{}.png'.format(m))
        imwrite(final_out, masks[m])

def save_one_mask(masks, output_path, mask_file_name):
    """
    :param masks: list of masked polygons from the mask_polygons_separately function 
    :param output_path: path to save the masks
    :param mask_file_name: the file name the masks should have 
    """
    # For each filled polygon, write the mask shape out to the file per image
    mask_file_name = path.join(output_path, mask_file_name + '.png')
    imwrite(mask_file_name, masks)
    

def read_json(json_path):
    """
    :param json_path: path to load json from
    :returns: a python dictionary of json features
    """
    annotations = json.load(open(json_path))
    return annotations


def get_feature_info(feature):
    """
    :param feature: a python dictionary of json labels
    :returns: a list mapping of polygons contained in the image 
    """
    # Getting each polygon points from the json file and adding it to a dictionary of uid:polygons
    props = {}

    for feat in feature['features']['xy']:
        feat_shape = wkt.loads(feat['wkt'])
        coords = list(mapping(feat_shape)['coordinates'][0])
        props[feat['properties']['uid']] = (np.array(coords, np.int32))

    return props


def mask_chips(json_path, images_directory, output_directory, single_file, border, pre_post='_post'):
    """
    :param json_path: path to find multiple json files for the chips
    :param images_directory: path to the directory containing the images to be masked
    :param output_directory: path to the directory where masks are to be saved
    :param single_file: a boolean value to see if masks should be saved a single file or multiple
    """
    # For each feature in the json we will create a separate mask
    # Getting all files in the directory provided for jsons
    jsons = [j for j in next(walk(json_path))[2] if pre_post in j]

    # After removing non-json items in dir (if any)
    for j in tqdm([j for j in jsons if j.endswith('json')],
                  unit='poly',
                  leave=False):
        # Our chips start off in life as PNGs
        chip_image_id = path.splitext(j)[0] + '.png'
        mask_file = path.splitext(j)[0]

        # Loading the per chip json
        j_full_path = path.join(json_path, j)
        chip_json = read_json(j_full_path)

        # Getting the full chip path, and loading the size dimensions
        chip_file = path.join(images_directory, chip_image_id)
        chip_size = get_dimensions(chip_file)

        # Reading in the polygons from the json file
        polys = get_feature_info(chip_json)

        # Getting a list of the polygons and saving masks as separate or single image files
        if len(polys) > 0:
            if single_file:
                if border > 0:
                    masked_polys = mask_polygons_together_with_border(chip_size, polys, border)
                else:
                    masked_polys = mask_polygons_together(chip_size, polys)
                save_one_mask(masked_polys, output_directory, mask_file)
            else:
                masked_polys = mask_polygons_separately(chip_size, polys)
                save_masks(masked_polys, output_directory, mask_file)

In [130]:
def create_masks(dataSplit='train', pre_post_str="post", single_file=True):
    # Parse command line arguments
    parser = argparse.ArgumentParser(
        description=
        """mask_polygons.py: Takes in xBD dataset and masks polygons in the image\n\n
        WARNING: This could lead to hundreds of output images per input\n""")
    
    parser.add_argument('--input',
                        required=True,
                        metavar="/path/to/xBD/",
                        help='Path to parent dataset directory "xBD"')
    parser.add_argument('--single-file', 
                        action='store_true',
                        help='use to save all masked polygon instances to a single file rather than one polygon per mask file')
    parser.add_argument('--border',
                        default=0,
                        type=int,
                        metavar="positive integer for pixel border (e.g. 1)",
                        help='Positive integer used to shrink the polygon by')
    
    if single_file: ### Creates all the masks in single file
        args = parser.parse_args(['--input', 'data/xview_building_damage/challenge/output' , '--single-file', '--border', '2'])
    else:
        args = parser.parse_args(['--input', 'data/xview_building_damage/challenge/'+dataSplit+'/disaster' , '--border', '2'])
    
    # Getting the list of the disaster types under the xBD directory
    disasters = next(walk(args.input))[1]
    
    for disaster in tqdm(disasters, desc='Masking', unit='disaster'):
        # Create the full path to the images, labels, and mask output directories
            print("disaster ", disaster)
            image_dir = path.join(args.input, disaster, 'images')
            json_dir = path.join(args.input, disaster, 'labels')
            output_dir = path.join(args.input, disaster, 'masks', pre_post_str)
        
            if not path.isdir(image_dir):
                print(
                    "Error, could not find image files in {}.\n\n"
                    .format(image_dir),
                    file=stderr)
                exit(2)
        
            if not path.isdir(json_dir):
                print(
                    "Error, could not find labels in {}.\n\n"
                    .format(json_dir),
                    file=stderr)
                exit(3)
        
            if not path.isdir(output_dir):
                makedirs(output_dir)
        
            mask_chips(json_dir, image_dir, output_dir, args.single_file, args.border, '_' + pre_post_str)

In [ ]:
create_masks('train', "post", single_file=False)

In [ ]:
create_masks('train', "post", single_file=False)  ## Completed Locally
create_masks('train', "pre", single_file=False)

In [ ]:
create_masks('hold', "pre", single_file=False)
create_masks('hold', "post", single_file=False)

In [ ]:
create_masks('test', "pre", single_file=False)
create_masks('test', "post", single_file=False)

### Spliting the images by disaster

#### Train

In [5]:
!python src/utils/mask_generators/split_into_disasters.py --input data/xview_building_damage/challenge/train/raw --output data/xview_building_damage/challenge/train/disaster

#### Hold

In [6]:
!python src/utils/mask_generators/split_into_disasters.py --input data/xview_building_damage/challenge/hold/raw --output data/xview_building_damage/challenge/hold/disaster

#### Test

In [7]:
!python src/utils/mask_generators/split_into_disasters.py --input data/xview_building_damage/challenge/test/raw --output data/xview_building_damage/challenge/test/disaster

### Overlay Masks and images Steps - Prepare for Pytorch Masks for Models

These steps are need to create the image classes for VIT and other models using Pytorch

### 1. Load the Metadata files

In [31]:
def load_meta_files(ROOT_DATA_DIR,disaster_name):
    train_meta_df=pd.read_csv(os.path.join(ROOT_DATA_DIR, 'challenge', 'csv', 'hc_train_'+disaster_name+'.csv'))
    valid_meta_df=pd.read_csv(os.path.join(ROOT_DATA_DIR, 'challenge', 'csv', 'hc_hold_'+disaster_name+'.csv'))
    test_meta_df=pd.read_csv(os.path.join(ROOT_DATA_DIR, 'challenge', 'csv' , 'hc_test_'+disaster_name+'.csv'))
    return train_meta_df, valid_meta_df, test_meta_df

In [14]:
def remove_pre_and_unclassified_class(df):
    data = df[df['damage'] != 'un-classified']
    data_wo_pre = data[data['damage'] != 'pre']
    return data_wo_pre

In [5]:
def clean_meta_file_wo_pre(disaster_name):
    train_meta_df, valid_meta_df, test_meta_df = load_meta_files(ROOT_DATA_DIR, disaster_name)
    train_data_wo_pre = remove_pre_and_unclassified_class(train_meta_df)
    valid_data_wo_pre = remove_pre_and_unclassified_class(valid_meta_df)
    test_data_wo_pre = remove_pre_and_unclassified_class(test_meta_df)
    return train_data_wo_pre, valid_data_wo_pre, test_data_wo_pre

### 2. Create mask and image overlay with cropped images

#### Creating polygon masks and overlay

In [17]:
def polygons_mask(polygons):
    img_mask = np.zeros(im_size, np.uint8)
    if not polygons:
        return img_mask
    int_coords = lambda x: np.array(x).round().astype(np.int32)
    exteriors = [int_coords(poly.exterior.coords) for poly in polygons]
    interiors = [int_coords(pi.coords) for poly in polygons
                 for pi in poly.interiors]
    cv2.fillPoly(img_mask, exteriors, 1)
    cv2.fillPoly(img_mask, interiors, 0)
    return img_mask

def create_image_mask_overall(root_dir, im_size, meta_df ,dataSplit = 'train', disaster_name='hurricane-michael', hurricane_name='michael'):
   input_dir =  os.path.join(root_dir, 'challenge', dataSplit, 'disaster', disaster_name) 
   img_input = os.path.join(input_dir, 'images')
   output_dir =os.path.join(root_dir, 'challenge', dataSplit, 'disaster','hurricanes-all', 'img_mask_overlay', hurricane_name )

   df = meta_df[meta_df['is_post_image'] == True]
   print("Starting : Mask overlay")
   for idx, file_name in enumerate(df['mask_file_names']):
       image = cv2.imread(os.path.join(img_input, df.iloc[idx]['img_name']))
       mask = np.zeros(image.shape[:2], dtype="uint8")
       _mask = polygons_mask([shapely.wkt.loads(df.iloc[idx]['image_polygon'])])
       masked = cv2.bitwise_and(image, image, mask=_mask)
       plt.imsave(os.path.join(output_dir, file_name), masked)
   print("Ending : Mask overlay")

#### Crop and save

In [44]:
def get_bounds_tp(image_wkt):
    bounds = wkt.loads(image_wkt).bounds
    return (bounds[0], bounds[1], bounds[2], bounds[3]) ## 

def crop_save_masked_images(data_split_dir, meta_df, crop_output_dir_name = 'img_mask_overlay_crops', disaster_name = 'hurricane-michael', hurricane_name='michael'):
    print("Starting Cropping the images")
    for idx, file_name in enumerate(meta_df['mask_file_names']):
        img = Image.open(os.path.join(data_split_dir, 'disaster', 'hurricanes-all','img_mask_overlay', hurricane_name, file_name))
        minx, miny, maxx, maxy = get_bounds_tp(meta_df.iloc[idx]['image_polygon'])
        cropped_img=img.crop((minx-5, miny-5, maxx+5, maxy+5))
        cropped_img.save(os.path.join(data_split_dir, 'disaster', 'hurricanes-all', crop_output_dir_name, hurricane_name, file_name))
    print("Finished Cropping the images")

In [47]:
def sort_masks_by_class(top_dir, meta_df, disaster_name='hurricane-all', hurricane_name='michael', post=True):
    root_path=os.path.join(top_dir, 'disaster', disaster_name)
    disas_post_mask= os.path.join(root_path,'img_mask_overlay_crops', hurricane_name) #source
    print("Source root : ", disas_post_mask)
    disas_class_path=os.path.join(root_path, 'img_mask_ov_crop_class')
    print("Destination root : ", disas_class_path)
    
    df = meta_df[meta_df['is_post_image'] == True]
    
    #    df = meta_df[meta_df['is_pre_image'] != post]
    print("Started moving the mask files to class folder for hurricane-", hurricane_name)
    for idx, file_name in enumerate(df['mask_file_names']):
        source = os.path.join(disas_post_mask, df.iloc[idx]['mask_file_names'])
        destination = os.path.join(disas_class_path, df.iloc[idx]['damage'])
        if os.path.exists(destination):
            pass
        else:
            print( "Creating dir for " , df.iloc[idx]['damage'])
            os.mkdir(destination)
        
        if os.path.exists(source):
            shutil.copy(source, destination)
    print("Finshed moving the mask files to class folder for hurricane-", hurricane_name)

#### Michael

In [ ]:
train_data_wo_pre,valid_data_wo_pre, test_data_wo_pre  = clean_meta_file_wo_pre('hurricane-michael')

In [ ]:
im_size =(1024, 1024)
create_image_mask_overall(ROOT_DATA_DIR, im_size, train_data_wo_pre, 'train', 'hurricane-michael', 'michael')
create_image_mask_overall(ROOT_DATA_DIR, im_size, valid_data_wo_pre, 'hold', 'hurricane-michael', 'michael')
create_image_mask_overall(ROOT_DATA_DIR, im_size, test_data_wo_pre, 'test', 'hurricane-michael', 'michael')

In [ ]:
crop_save_masked_images(TRAIN_DIR, train_data_wo_pre, crop_output_dir_name = 'img_mask_overlay_crops', disaster_name = 'hurricane-michael', hurricane_name='michael')
crop_save_masked_images(HOLD_DIR, valid_data_wo_pre, crop_output_dir_name = 'img_mask_overlay_crops', disaster_name = 'hurricane-michael', hurricane_name='michael')
crop_save_masked_images(TEST_DIR, test_data_wo_pre, crop_output_dir_name = 'img_mask_overlay_crops', disaster_name = 'hurricane-michael', hurricane_name='michael')

In [ ]:
sort_masks_by_class(TRAIN_DIR , train_data_wo_pre , 'hurricanes-all',  hurricane_name='michael')
sort_masks_by_class(HOLD_DIR , valid_data_wo_pre , 'hurricanes-all',  hurricane_name='michael')
sort_masks_by_class(TEST_DIR , test_data_wo_pre , 'hurricanes-all',  hurricane_name='michael')

#### Florence

In [33]:
train_data_wo_pre,valid_data_wo_pre, test_data_wo_pre  = clean_meta_file_wo_pre('hurricane-florence')

In [34]:
train_data_wo_pre

,Unnamed: 0,sensor,provider_asset_type,gsd,capture_date,off_nadir_angle,pan_resolution,sun_azimuth,sun_elevation,target_azimuth,...,building_id,image_polygon,feature_type,damage,dataset,image_id,is_pre_image,is_post_image,mask_file_names,damage_class
110,221034,GEOEYE01,GEOEYE01,2.895928,2018-09-18 16:32:47+00:00,41.741340,0.722669,162.69955,57.024815,69.02214,...,b4e1ef10-eda9-4879-87c2-6f0025b861b4,"POLYGON ((404.8428229479346 1022.985470946143,...",building,no-damage,train,hurricane-florence_00000001,False,True,hurricane-florence_00000001_post_disaster_b4e1...,1
111,221035,GEOEYE01,GEOEYE01,2.895928,2018-09-18 16:32:47+00:00,41.741340,0.722669,162.69955,57.024815,69.02214,...,b7e5d853-6fee-41c8-b5c7-4ea5d7f7a296,"POLYGON ((379.9585035544198 1004.636700643454,...",building,no-damage,train,hurricane-florence_00000001,False,True,hurricane-florence_00000001_post_disaster_b7e5...,1
112,221036,GEOEYE01,GEOEYE01,2.895928,2018-09-18 16:32:47+00:00,41.741340,0.722669,162.69955,57.024815,69.02214,...,742703de-5862-4aa6-b739-ba05b0598021,"POLYGON ((333.5734307474256 971.2194976514633,...",building,no-damage,train,hurricane-florence_00000001,False,True,hurricane-florence_00000001_post_disaster_7427...,1
113,221037,GEOEYE01,GEOEYE01,2.895928,2018-09-18 16:32:47+00:00,41.741340,0.722669,162.69955,57.024815,69.02214,...,0ada4abf-2075-4c90-95ba-dc4e24fbbc0c,"POLYGON ((313.4536771412606 944.5107167134208,...",building,no-damage,train,hurricane-florence_00000001,False,True,hurricane-florence_00000001_post_disaster_0ada...,1
114,221038,GEOEYE01,GEOEYE01,2.895928,2018-09-18 16:32:47+00:00,41.741340,0.722669,162.69955,57.024815,69.02214,...,09741acc-1b6b-4183-8d32-0329ff917003,"POLYGON ((85.82098497048729 996.4078882593402,...",building,no-damage,train,hurricane-florence_00000001,False,True,hurricane-florence_00000001_post_disaster_0974...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12887,233837,GEOEYE01,GEOEYE01,2.091625,2018-09-20 16:04:41+00:00,28.017313,0.522825,153.94543,53.722378,190.82309,...,1ab6de84-c28c-4600-90ae-747ac750b1f7,"POLYGON ((338.9242822473159 824.4681114714469,...",building,no-damage,train,hurricane-florence_00000545,False,True,hurricane-florence_00000545_post_disaster_1ab6...,1
12888,233838,GEOEYE01,GEOEYE01,2.091625,2018-09-20 16:04:41+00:00,28.017313,0.522825,153.94543,53.722378,190.82309,...,8586d216-5f4d-4d6b-b888-d3aab23d50d9,"POLYGON ((528.3778450679621 684.4573814955489,...",building,no-damage,train,hurricane-florence_00000545,False,True,hurricane-florence_00000545_post_disaster_8586...,1
12889,233839,GEOEYE01,GEOEYE01,2.091625,2018-09-20 16:04:41+00:00,28.017313,0.522825,153.94543,53.722378,190.82309,...,494ffde0-dbe4-4c18-a81f-c77ae6f461bb,"POLYGON ((802.7019802489498 641.6557056870366,...",building,no-damage,train,hurricane-florence_00000545,False,True,hurricane-florence_00000545_post_disaster_494f...,1
12890,233840,GEOEYE01,GEOEYE01,2.091625,2018-09-20 16:04:41+00:00,28.017313,0.522825,153.94543,53.722378,190.82309,...,7aa9c634-7f37-4555-90ef-20fdcee1eb72,"POLYGON ((105.8794847856132 234.6895093332477,...",building,no-damage,train,hurricane-florence_00000545,False,True,hurricane-florence_00000545_post_disaster_7aa9...,1


In [39]:
im_size =(1024, 1024)
create_image_mask_overall(ROOT_DATA_DIR, im_size, train_data_wo_pre, 'train', 'hurricane-florence', 'florence')

Starting : Mask overlay
Ending : Mask overlay


In [41]:
create_image_mask_overall(ROOT_DATA_DIR, im_size, valid_data_wo_pre, 'hold', 'hurricane-florence', 'florence')

Starting : Mask overlay
Ending : Mask overlay


In [43]:
create_image_mask_overall(ROOT_DATA_DIR, im_size, test_data_wo_pre, 'test', 'hurricane-florence', 'florence')

Starting : Mask overlay
Ending : Mask overlay


In [46]:
crop_save_masked_images(TRAIN_DIR, train_data_wo_pre, crop_output_dir_name = 'img_mask_overlay_crops', disaster_name = 'hurricane-florence', hurricane_name='florence')

Starting Cropping the images
Finished Cropping the images


In [51]:
crop_save_masked_images(HOLD_DIR, valid_data_wo_pre, crop_output_dir_name = 'img_mask_overlay_crops', disaster_name = 'hurricane-florence', hurricane_name='florence')

Starting Cropping the images
Finished Cropping the images


In [52]:
crop_save_masked_images(TEST_DIR, test_data_wo_pre, crop_output_dir_name = 'img_mask_overlay_crops', disaster_name = 'hurricane-florence', hurricane_name='florence')

Starting Cropping the images
Finished Cropping the images


In [49]:
sort_masks_by_class(TRAIN_DIR , train_data_wo_pre , 'hurricanes-all',  hurricane_name='florence')

Source root :  /Users/yaminigotimukul/DataScience/Berekley/Semesters/Spring_2024/repo/alivio/data/xview_building_damage/challenge/train/disaster/hurricanes-all/img_mask_overlay_crops/florence
Destination root :  /Users/yaminigotimukul/DataScience/Berekley/Semesters/Spring_2024/repo/alivio/data/xview_building_damage/challenge/train/disaster/hurricanes-all/img_mask_ov_crop_class
Started moving the mask files to class folder for hurricane- florence
Finshed moving the mask files to class folder for hurricane- florence


In [53]:
sort_masks_by_class(HOLD_DIR , valid_data_wo_pre , 'hurricanes-all',  hurricane_name='florence')

Source root :  /Users/yaminigotimukul/DataScience/Berekley/Semesters/Spring_2024/repo/alivio/data/xview_building_damage/challenge/hold/disaster/hurricanes-all/img_mask_overlay_crops/florence
Destination root :  /Users/yaminigotimukul/DataScience/Berekley/Semesters/Spring_2024/repo/alivio/data/xview_building_damage/challenge/hold/disaster/hurricanes-all/img_mask_ov_crop_class
Started moving the mask files to class folder for hurricane- florence
Finshed moving the mask files to class folder for hurricane- florence


In [54]:
sort_masks_by_class(TEST_DIR , test_data_wo_pre , 'hurricanes-all',  hurricane_name='florence')

Source root :  /Users/yaminigotimukul/DataScience/Berekley/Semesters/Spring_2024/repo/alivio/data/xview_building_damage/challenge/test/disaster/hurricanes-all/img_mask_overlay_crops/florence
Destination root :  /Users/yaminigotimukul/DataScience/Berekley/Semesters/Spring_2024/repo/alivio/data/xview_building_damage/challenge/test/disaster/hurricanes-all/img_mask_ov_crop_class
Started moving the mask files to class folder for hurricane- florence
Finshed moving the mask files to class folder for hurricane- florence


#### Harvey

In [57]:
train_data_wo_pre,valid_data_wo_pre, test_data_wo_pre  = clean_meta_file_wo_pre('hurricane-harvey')

In [59]:
im_size =(1024, 1024)
create_image_mask_overall(ROOT_DATA_DIR, im_size, train_data_wo_pre, 'train', 'hurricane-harvey', 'harvey')
create_image_mask_overall(ROOT_DATA_DIR, im_size, valid_data_wo_pre, 'hold', 'hurricane-harvey', 'harvey')
create_image_mask_overall(ROOT_DATA_DIR, im_size, test_data_wo_pre, 'test', 'hurricane-harvey', 'harvey')

Starting : Mask overlay
Ending : Mask overlay
Starting : Mask overlay
Ending : Mask overlay
Starting : Mask overlay
Ending : Mask overlay


In [60]:
crop_save_masked_images(TRAIN_DIR, train_data_wo_pre, crop_output_dir_name = 'img_mask_overlay_crops', disaster_name = 'hurricane-harvey', hurricane_name='harvey')
crop_save_masked_images(HOLD_DIR, valid_data_wo_pre, crop_output_dir_name = 'img_mask_overlay_crops', disaster_name = 'hurricane-harvey', hurricane_name='harvey')
crop_save_masked_images(TEST_DIR, test_data_wo_pre, crop_output_dir_name = 'img_mask_overlay_crops', disaster_name = 'hurricane-harvey', hurricane_name='harvey')

Starting Cropping the images
Finished Cropping the images
Starting Cropping the images
Finished Cropping the images
Starting Cropping the images
Finished Cropping the images


In [61]:
sort_masks_by_class(TRAIN_DIR , train_data_wo_pre , 'hurricanes-all',  hurricane_name='harvey')
sort_masks_by_class(HOLD_DIR , valid_data_wo_pre , 'hurricanes-all',  hurricane_name='harvey')
sort_masks_by_class(TEST_DIR , test_data_wo_pre , 'hurricanes-all',  hurricane_name='harvey')

Source root :  /Users/yaminigotimukul/DataScience/Berekley/Semesters/Spring_2024/repo/alivio/data/xview_building_damage/challenge/train/disaster/hurricanes-all/img_mask_overlay_crops/harvey
Destination root :  /Users/yaminigotimukul/DataScience/Berekley/Semesters/Spring_2024/repo/alivio/data/xview_building_damage/challenge/train/disaster/hurricanes-all/img_mask_ov_crop_class
Started moving the mask files to class folder for hurricane- harvey
Finshed moving the mask files to class folder for hurricane- harvey
Source root :  /Users/yaminigotimukul/DataScience/Berekley/Semesters/Spring_2024/repo/alivio/data/xview_building_damage/challenge/hold/disaster/hurricanes-all/img_mask_overlay_crops/harvey
Destination root :  /Users/yaminigotimukul/DataScience/Berekley/Semesters/Spring_2024/repo/alivio/data/xview_building_damage/challenge/hold/disaster/hurricanes-all/img_mask_ov_crop_class
Started moving the mask files to class folder for hurricane- harvey
Finshed moving the mask files to class fol

#### Matthew

In [64]:
train_data_wo_pre,valid_data_wo_pre, test_data_wo_pre  = clean_meta_file_wo_pre('hurricane-matthew')

In [65]:
im_size =(1024, 1024)
create_image_mask_overall(ROOT_DATA_DIR, im_size, train_data_wo_pre, 'train', 'hurricane-matthew', 'matthew')
create_image_mask_overall(ROOT_DATA_DIR, im_size, valid_data_wo_pre, 'hold', 'hurricane-matthew', 'matthew')
create_image_mask_overall(ROOT_DATA_DIR, im_size, test_data_wo_pre, 'test', 'hurricane-matthew', 'matthew')

Starting : Mask overlay
Ending : Mask overlay
Starting : Mask overlay
Ending : Mask overlay
Starting : Mask overlay
Ending : Mask overlay


In [66]:
crop_save_masked_images(TRAIN_DIR, train_data_wo_pre, crop_output_dir_name = 'img_mask_overlay_crops', disaster_name = 'hurricane-matthew', hurricane_name='matthew')
crop_save_masked_images(HOLD_DIR, valid_data_wo_pre, crop_output_dir_name = 'img_mask_overlay_crops', disaster_name = 'hurricane-matthew', hurricane_name='matthew')
crop_save_masked_images(TEST_DIR, test_data_wo_pre, crop_output_dir_name = 'img_mask_overlay_crops', disaster_name = 'hurricane-matthew', hurricane_name='matthew')

Starting Cropping the images
Finished Cropping the images
Starting Cropping the images
Finished Cropping the images
Starting Cropping the images
Finished Cropping the images


In [67]:
sort_masks_by_class(TRAIN_DIR , train_data_wo_pre , 'hurricanes-all',  hurricane_name='matthew')
sort_masks_by_class(HOLD_DIR , valid_data_wo_pre , 'hurricanes-all',  hurricane_name='matthew')
sort_masks_by_class(TEST_DIR , test_data_wo_pre , 'hurricanes-all',  hurricane_name='matthew')

Source root :  /Users/yaminigotimukul/DataScience/Berekley/Semesters/Spring_2024/repo/alivio/data/xview_building_damage/challenge/train/disaster/hurricanes-all/img_mask_overlay_crops/matthew
Destination root :  /Users/yaminigotimukul/DataScience/Berekley/Semesters/Spring_2024/repo/alivio/data/xview_building_damage/challenge/train/disaster/hurricanes-all/img_mask_ov_crop_class
Started moving the mask files to class folder for hurricane- matthew
Finshed moving the mask files to class folder for hurricane- matthew
Source root :  /Users/yaminigotimukul/DataScience/Berekley/Semesters/Spring_2024/repo/alivio/data/xview_building_damage/challenge/hold/disaster/hurricanes-all/img_mask_overlay_crops/matthew
Destination root :  /Users/yaminigotimukul/DataScience/Berekley/Semesters/Spring_2024/repo/alivio/data/xview_building_damage/challenge/hold/disaster/hurricanes-all/img_mask_ov_crop_class
Started moving the mask files to class folder for hurricane- matthew
Finshed moving the mask files to clas